In [1]:
import csv
import time
from extratools.dicttools import nextentries

def importCSV(filename):
#read CSV file
    with open(input_file, newline='') as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            data_set.append(row)
def exportCSV(filename):
    # write CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile,delimiter=' ')  
        
        KI=[]
        VI=[]
        for item in support_data:
            KI.append(item[1])
            VI.append(item[0])
        
            temp=list(zip(KI,VI))
            temp.sort()
            KI[:],VI[:]=zip(*temp)
        #"""
        for i in range(len(KI)):
            processkey=str(KI[i]).replace("'","").replace("(","[").replace(")","]").replace(" ","")
            writer.writerow('{key},{value}'.format(key =processkey , value = VI[i]))
        #"""

In [2]:
class PrefixSpan(object):
    def __init__(self, db):
        # type: (List[List[int]]) -> None
        self._db = db

        self.minlen, self.maxlen = 1, 1000

        self._results = [] # type: Any

    def _mine(self, func):
        # type: (Callable[[Pattern, Matches], None]) -> Any
        self._results.clear()

        func([], [(i, -1) for i in range(len(self._db))])

        return self._results

    frequent = None # type: Callable
    topk = None # type: Callable

    defaultkey = lambda patt, matches: len(matches)

In [3]:
def PrefixSpan_frequent(
        self, minsup, closed=False, generator=False,
        key=None, bound=None,
        filter=None, callback=None
    ):
    # type: (PrefixSpan, int, bool, bool, Optional[Key], Optional[Key], Optional[Filter], Optional[Callback]) -> Results
    if generator:
        occursstack = [] # type: List[Occurs]

    def canpass(sup):
        # type: (int) -> bool
        return sup < minsup

    def verify(patt, matches):
        # type: (Pattern, Matches) -> None
        sup = key(patt, matches)
        if canpass(sup):
            return

        if (filter is None or filter(patt, matches)) and (
                (not closed or isclosed(self._db, patt, matches)) and
                (not generator or isgenerator(self._db, patt, matches, occursstack))
            ):
            if callback:
                callback(patt, matches)
            else:
                self._results.append((sup, patt))

    def frequent_rec(patt, matches):
        # type: (Pattern, Matches) -> None
        if len(patt) >= self.minlen:
            verify(patt, matches)

            if len(patt) == self.maxlen:
                return

        occurs = nextentries(self._db, matches)
        if generator:
            occursstack.append(occurs)

        for newitem, newmatches in occurs.items():
            newpatt = patt + [newitem]
            if canpass(bound(newpatt, newmatches)) or (
                    closed and canclosedprune(self._db, newpatt, newmatches) or
                    generator and cangeneratorprune(self._db, newpatt, newmatches, occursstack)
                ):
                continue

            frequent_rec(newpatt, newmatches)

        if generator:
            occursstack.pop()

    if key is None:
        key = bound = PrefixSpan.defaultkey

    results = self._mine(frequent_rec)

    return None if callback else results

In [4]:
def PrefixSpan_topk(
        self, k, closed=False, generator=False,
        key=None, bound=None,
        filter=None, callback=None
    ):
    # type: (PrefixSpan, int, bool, bool, Optional[Key], Optional[Key], Optional[Filter], Optional[Callback]) -> Results
    if generator:
        occursstack = [] # type: List[Occurs]

    def canpass(sup):
        # type: (int) -> bool
        return len(self._results) == k and sup <= self._results[0][0]


    def verify(patt, matches):
        # type: (Pattern, Matches) -> None
        sup = key(patt, matches)
        if canpass(sup):
            return

        if (filter is None or filter(patt, matches)) and (
                (not closed or isclosed(self._db, patt, matches)) and
                (not generator or isgenerator(self._db, patt, matches, occursstack))
            ):
            (heappush if len(self._results) < k else heappushpop)(self._results, (sup, patt, matches))

    def topk_rec(patt, matches):
        # type: (Pattern, Matches) -> None
        if len(patt) >= self.minlen:
            verify(patt, matches)

            if len(patt) == self.maxlen:
                return

        occurs = nextentries(self._db, matches)
        if generator:
            occursstack.append(occurs)

        for newitem, newmatches in sorted(
                occurs.items(),
                key=lambda x: key(patt + [x[0]], x[1]),
                reverse=True
            ):
            newpatt = patt + [newitem]

            if canpass(bound(newpatt, newmatches)):
                break

            if (
                    closed and canclosedprune(self._db, newpatt, newmatches) or
                    generator and cangeneratorprune(self._db, newpatt, newmatches, occursstack)
                ):
                continue

            topk_rec(newpatt, newmatches)

        if generator:
            occursstack.pop()

    if key is None:
        key = bound = PrefixSpan.defaultkey

    # Sort by support in reverse, then by pattern.
    results = sorted(self._mine(topk_rec), key=lambda x: (-x[0], x[1]))

    if callback:
        for _, patt, matches in results:
            callback(patt, matches)

        return None

    return [(sup, patt) for sup, patt, _ in results]

In [5]:
PrefixSpan.frequent = PrefixSpan_frequent
PrefixSpan.topk = PrefixSpan_topk

In [6]:
# dataset_1
tStart = time.time()
data_set = []

temp=[]

input_file='dataset_3.csv'
output_file='output_1.csv'
support=2

importCSV(input_file)
ps = PrefixSpan(data_set)
support_data=ps.frequent(support)
exportCSV(output_file)

tEnd = time.time()
print ("It cost %f sec"%(tEnd - tStart))
print (tEnd - tStart)

It cost 0.038002 sec
0.03800201416015625


In [7]:
# dataset_2
tStart = time.time()
data_set = []

temp=[]

input_file='dataset_3.csv'
output_file='output_1.csv'
support=2

importCSV(input_file)
ps = PrefixSpan(data_set)
support_data=ps.frequent(support)
exportCSV(output_file)

tEnd = time.time()
print ("It cost %f sec"%(tEnd - tStart))
print (tEnd - tStart)

It cost 0.051003 sec
0.05100297927856445


In [8]:
# dataset_3
tStart = time.time()
data_set = []

temp=[]

input_file='dataset_3.csv'
output_file='output_1.csv'
support=2

importCSV(input_file)
ps = PrefixSpan(data_set)
support_data=ps.frequent(support)
exportCSV(output_file)

tEnd = time.time()
print ("It cost %f sec"%(tEnd - tStart))
print (tEnd - tStart)

It cost 0.040002 sec
0.04000234603881836


In [9]:
# dataset_4
tStart = time.time()
data_set = []

temp=[]

input_file='dataset_3.csv'
output_file='output_1.csv'
support=3

importCSV(input_file)
ps = PrefixSpan(data_set)
support_data=ps.frequent(support)
exportCSV(output_file)

tEnd = time.time()
print ("It cost %f sec"%(tEnd - tStart))
print (tEnd - tStart)

It cost 0.012001 sec
0.012000799179077148


In [10]:
# dataset_5
tStart = time.time()
data_set = []

temp=[]

input_file='dataset_3.csv'
output_file='output_1.csv'
support=4

importCSV(input_file)
ps = PrefixSpan(data_set)
support_data=ps.frequent(support)
exportCSV(output_file)

tEnd = time.time()
print ("It cost %f sec"%(tEnd - tStart))
print (tEnd - tStart)

It cost 0.007000 sec
0.007000446319580078


In [11]:
# dataset_6
tStart = time.time()
data_set = []

temp=[]

input_file='dataset_3.csv'
output_file='output_1.csv'
support=4

importCSV(input_file)
ps = PrefixSpan(data_set)
support_data=ps.frequent(support)
exportCSV(output_file)

tEnd = time.time()
print ("It cost %f sec"%(tEnd - tStart))
print (tEnd - tStart)

It cost 0.009001 sec
0.009000778198242188
